# 1. Semantic Annotation

## 1.1 Using Tagme


- Tagme demo (GUI): https://tagme.d4science.org/tagme/
- To create account https://sobigdata.d4science.org/group/tagme/tagme


In [1]:
import requests
token = 'cf3c6c6c-c6ea-4ab9-8a70-3a558775e837-843339462' # colocar aquí el token
url_endpoint = 'https://tagme.d4science.org/tagme/tag?lang=en&include_abstract=true&include_categories=true&gcube-token='
headers = {'user-agent': 'Mozilla/5.0', 'accept': 'application/json', 'content-type': 'application/json'}
dbr = 'http://dbpedia.org/resource/'
dbc = 'http://dbpedia.org/resource/Category:'


### 1.1.1 To process title or abstract:

Try with:

https://tagme.d4science.org/tagme/spot?lang=en&gcube-token=<PUT YOUR TOKEN>&tweet=true&text=We%20propose%20a%20fact%20validation%20challenge%20for%20ISWC%202019.%20The%20participants%20will%20be%20provided%20set%20of%20facts%20in%20the%20form%20of%20a%20set%20of%20RDF%20triples%20as%20input.%20They%20will%20be%20requested%20to%20evaluate%20the%20veracity%20of%20this%20input%20and%20return%20a%20score%20between%20-1%20(false)%20and%20+1%20(true).%20This%20score%20%20quantifies%20how%20reliable%20the%20given%20facts%20are.%20The%20challenge%20will%20be%20based%20on%20the%20biomedical%20knowledge%20graph%20DrugBank%20and%20consist%20of%20two%20tasks.%20In%20Task%201%20(Batch%20Fact%20Validation),%20the%20ROC%20achieved%20by%20the%20participants%20will%20be%20evaluated%20using%20the%20GERBIL%20extension%20used%20successfully%20during%20the%20ISWC%20Semantic%20Web%20Challenges%202017%20and%202018.%20Task%202%20will%20be%20concerned%20with%20fact%20validation%20at%20scale%20and%20rely%20on%20the%20HOBBIT%20benchmarking%20platform.%20Here,%20the%20participants%20will%20be%20faced%20with%20an%20interactive%20task,%20in%20which%20both%20ROC%20and%20runtime%20will%20be%20evaluated.

In [2]:

def getAnnotations(paper_id, text):
    url = url_endpoint + token + '&text=' + text

    resp = requests.get(url, headers=headers).json()
    resp.keys()
    annotations = resp['annotations']

    dbCategories = []

    for i in range(len(annotations)):
        ann = annotations[i]['spot']
        if annotations[i]['rho'] > 0.1 and annotations[i]['link_probability'] > 0.1:
            ann = dbr + ann.replace(' ', '_')     
            # Guardar como recursos de la Dbpedia:
            for c in annotations[i]['dbpedia_categories']:
                dbCategories.append([paper_id, ann, annotations[i]['rho'],
                                 annotations[i]['link_probability'], dbc + c.replace(' ', '_')])
    return dbCategories

In [3]:


# To get annotations in a iterative way from the source:
#paper_id = 'http://dx.doi.org/10.14213/inteuniorigh.27.3.0020' # id or some identifier
#text = """We propose a fact validation challenge for ISWC 2019. The participants will be provided set of facts in the form of a set of RDF triples as input. They will be requested to evaluate the veracity of this input and return a score between -1 (false) and +1 (true). This score  quantifies how reliable the given facts are. The challenge will be based on the biomedical knowledge graph DrugBank and consist of two tasks. In Task 1 (Batch Fact Validation), the ROC achieved by the participants will be evaluated using the GERBIL extension used successfully during the ISWC Semantic Web Challenges 2017 and 2018. Task 2 will be concerned with fact validation at scale and rely on the HOBBIT benchmarking platform. Here, the participants will be faced with an interactive task, in which both ROC and runtime will be evaluated."""
#dbCategories = getAnnotations(paper_id, text)
with open("dois.txt", "r") as tf:
    lines = tf.read().split(',')
    for line in lines:
        # To get annotations in a iterative way from the source:
        paper_id = line # id or some identifier          
        text = """We propose a fact validation challenge for ISWC 2019. The participants will be provided set of facts in the form of a set of RDF triples as input. They will be requested to evaluate the veracity of this input and return a score between -1 (false) and +1 (true). This score  quantifies how reliable the given facts are. The challenge will be based on the biomedical knowledge graph DrugBank and consist of two tasks. In Task 1 (Batch Fact Validation), the ROC achieved by the participants will be evaluated using the GERBIL extension used successfully during the ISWC Semantic Web Challenges 2017 and 2018. Task 2 will be concerned with fact validation at scale and rely on the HOBBIT benchmarking platform. Here, the participants will be faced with an interactive task, in which both ROC and runtime will be evaluated."""
        dbCategories = getAnnotations(paper_id, text)
        fic = open("text_4.txt", "a+")
        fic.writelines("%s\n" % s for s in dbCategories)

fic.close()

In [4]:
# Explore the results:

for a in dbCategories:
    print(40*'-')
    print(a) # DBpedia categories

----------------------------------------
["\n'102307jctv1850gc423'", 'http://dbpedia.org/resource/ISWC', 0.24496769905090332, 0.31707316637039185, 'http://dbpedia.org/resource/Category:Music_technology']
----------------------------------------
["\n'102307jctv1850gc423'", 'http://dbpedia.org/resource/ISWC', 0.24496769905090332, 0.31707316637039185, 'http://dbpedia.org/resource/Category:ISO_standards']
----------------------------------------
["\n'102307jctv1850gc423'", 'http://dbpedia.org/resource/ISWC', 0.24496769905090332, 0.31707316637039185, 'http://dbpedia.org/resource/Category:Identifiers']
----------------------------------------
["\n'102307jctv1850gc423'", 'http://dbpedia.org/resource/ISWC', 0.24496769905090332, 0.31707316637039185, 'http://dbpedia.org/resource/Category:Universal_identifiers']
----------------------------------------
["\n'102307jctv1850gc423'", 'http://dbpedia.org/resource/RDF', 0.3218223750591278, 0.4697193503379822, 'http://dbpedia.org/resource/Category:Resou

###  Next steps (to do):

- Manual analysis to set the most suitable threshold for rho and link_probability.
- Filter results acccording thresholds:
- Enrich the KG with new triplets. For example:
    - Connecting each paper to DBPedia resources:
    <http://data.example.org/1101D5345H58> schema:mentions <http://dbpedia.org/resource/RDF>
    
    - Connecting each paper to DBpedia categories: 
    <http://data.example.org/1101D5345H58> dct:subject <http://dbpedia.org/resource/Category:Semantic_Web> .

- To save in your KG.
    
    

### 1.1.2 To find equivalent topics (keywords):

In [5]:
#keyword = 'Knowledge extraction'
keyword = 'Bibliographic Metadata'
paper_id = 'http://data.example.org/1101D5345H58' # id or some identifier

dbCategoriesK = getAnnotations(paper_id, keyword)

In [6]:
# Explore the results:

for a in dbCategoriesK:
    print(40*'-')
    print(a) # DBpedia categories
    
    
# Saving results according the last steps.

----------------------------------------
['http://data.example.org/1101D5345H58', 'http://dbpedia.org/resource/Metadata', 0.45757266879081726, 0.49475109577178955, 'http://dbpedia.org/resource/Category:Data_management']
----------------------------------------
['http://data.example.org/1101D5345H58', 'http://dbpedia.org/resource/Metadata', 0.45757266879081726, 0.49475109577178955, 'http://dbpedia.org/resource/Category:Knowledge_representation']
----------------------------------------
['http://data.example.org/1101D5345H58', 'http://dbpedia.org/resource/Metadata', 0.45757266879081726, 0.49475109577178955, 'http://dbpedia.org/resource/Category:Library_cataloging_and_classification']
----------------------------------------
['http://data.example.org/1101D5345H58', 'http://dbpedia.org/resource/Metadata', 0.45757266879081726, 0.49475109577178955, 'http://dbpedia.org/resource/Category:Metadata']
----------------------------------------
['http://data.example.org/1101D5345H58', 'http://dbpedi

## 1.2. Using DBpedia Spotlight



- Home page: https://www.dbpedia-spotlight.org

- API Documentation: 


- Way 1: using the pyspotlight library of Python: pip install pyspotlight

    https://pypi.org/project/pyspotlight/
    
    
    
- Way 2: using the spotligh library of Python: pip install spacy-dbpedia-spotlight

    https://pypi.org/project/spacy-dbpedia-spotlight/

In [7]:
import spotlight

results = spotlight.annotate('https://api.dbpedia-spotlight.org/en/annotate', text, confidence=0.5, 
support=20)
results

ModuleNotFoundError: No module named 'spotlight'

In [ ]:
# Querying using the API Endpoint (for keywords):
S = requests.Session()
headers = {'User-agent': 'Mozilla/5.0'}
r = S.get(url='https://api.dbpedia-spotlight.org/en/annotate?text=' + keyword)
# r.headers

In [ ]:
if r.status_code == 200:
    print(r.content)

# 2. Retrieving more data from DBpedia Sparq Endpoint



### Requirements:

- Install SPARQLWrapper on a pyton environment.
- Get a list of unique DBPedia instances and DBPedia concepts.

In [ ]:
dbrUnique = set([r[1] for r in dbCategories])
dbrUnique

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON, RDF, XML
# endpoint:
e = "http://dbpedia.org/sparql"

# Query template:
Q = """SELECT DISTINCT ?prop ?value WHERE {
  VALUES ?root {<%s>}
  VALUES ?prop {%s}
  ?root ?prop ?value .
  FILTER(LANG(?value)='en')
} ORDER BY ?value"""


In [ ]:
def getTriplets(node, q, p):
    sparql = SPARQLWrapper(e)
    sparql.setQuery(q%(node, p))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    for result in results["results"]["bindings"]:
        print(node, result['prop']["value"], result['value']["value"]) # To save in your KG

# To run from a unique list or dbr:
for r in dbrUnique:
    getTriplets(r, Q, 'rdfs:label')
